In [1]:
import pulp

## Formular el modelo de transporte asociado

# Refinerías

Existen 3 refinerías lo cual formamos el conjunto de i = {1,2,3} que es refinería 1,2,3
Existen 3 áreas de distribución por lo cual definimos j = {1,2,3} que pertenece a área de distribución 1,2,3

# Parámetros
Para cada refinería, tiene su capacidad diaria lo cual lo denotariamos así
Si , donde i es el conjunto de refinerías y S como capacidad

Tambien tiene su Demanda diaria de millones de galones donde lo definimos así:
Dj, donde D es demanda y j es el conjunto del área de distribución.

Por otro lado existe el parámetro de costo de tranportar los millones de galones desde la refinería hasta el área de distribución:
Cij donde C es el costo de tranporte de refinería i hasta área de distribución j

Por otro lado hay que tomar en cuenta la cantidad de millones de galones que se puede transportar y que no sea menor a 0


\begin{align*}

\textbf{Función objetivo:} \quad & \min \sum_{i=1}^3 \sum_{j=1}^3 c_{ij} \cdot x_{ij} \\
\\
\textbf{Sujeto a:} \quad 
& \sum_{j=1}^3 x_{ij} \leq S_i \quad \forall i \in \{1,2,3\} \quad \text{(Capacidad de refinería)} \\
& \sum_{i=1}^3 x_{ij} = D_j \quad \forall j \in \{1,2,3\} \quad \text{(Demanda que hay en esta área)} \\
\end{align*}


Tomar en cuenta que 
x13 no tiene conexión

In [ ]:

refinerias = [1, 2, 3]
areas = [1, 2, 3]

oferta = {1: 6, 2: 5, 3: 8}
demanda = {1: 4, 2: 8, 3: 7}

# Distancias (en km)
distancias = {
    (1,1): 120, (1,2): 180,           # R1 no se conecta a A3
    (2,1): 300, (2,2): 100, (2,3): 80,
    (3,1): 200, (3,2): 250, (3,3): 120
}

costo = {k: distancias[k] * 0.10 for k in distancias}

prob = pulp.LpProblem("TransporteGasolina", pulp.LpMinimize)

x = pulp.LpVariable.dicts("x", costo.keys(), lowBound=0)

prob += pulp.lpSum([costo[i,j] * x[i,j] for (i,j) in costo])

for i in refinerias:
    prob += pulp.lpSum([x[i,j] for j in areas if (i,j) in x]) <= oferta[i]

for j in areas:
    prob += pulp.lpSum([x[i,j] for i in refinerias if (i,j) in x]) == demanda[j]

prob.solve()

print("Costo total: $", pulp.value(prob.objective))
for (i,j) in x:
    if x[i,j].varValue > 0:
        print(f"Enviar {x[i,j].varValue:.2f} millones de galones de R{i} a A{j}")


Costo total: $ 243.0
Enviar 4.00 millones de galones de R1 a A1
Enviar 2.00 millones de galones de R1 a A2
Enviar 5.00 millones de galones de R2 a A2
Enviar 1.00 millones de galones de R3 a A2
Enviar 7.00 millones de galones de R3 a A3


In [ ]:

refinerias = [1, 2, 3]
areas = [1, 2, 3]

oferta = {1: 6, 2: 5, 3: 8}
demanda = {1: 4, 2: 8, 3: 4}  # A3 disminuyó

distancias = {
    (1,1): 120, (1,2): 180,           # R1 no se conecta a A3
    (2,1): 300, (2,2): 100, (2,3): 80,
    (3,1): 200, (3,2): 250, (3,3): 120
}

costo = {k: distancias[k] * 0.10 for k in distancias}

prob2 = pulp.LpProblem("TransporteGasolina_c", pulp.LpMinimize)

x = pulp.LpVariable.dicts("x", costo.keys(), lowBound=0)

exceso = {
    1: pulp.LpVariable("exceso_1", lowBound=0),
    2: pulp.LpVariable("exceso_2", lowBound=0),
    3: pulp.LpVariable("exceso_3", lowBound=0)
}

prob2 += pulp.lpSum([costo[i,j] * x[i,j] for (i,j) in x]) + \
         15000 * exceso[1] + 22000 * exceso[2]  # R3 sin costo

for i in refinerias:
    prob2 += pulp.lpSum([x[i,j] for j in areas if (i,j) in x]) + exceso[i] == oferta[i]

for j in areas:
    prob2 += pulp.lpSum([x[i,j] for i in refinerias if (i,j) in x]) == demanda[j]

prob2.solve()

print("Costo total: $", pulp.value(prob2.objective))
for (i,j) in x:
    if x[i,j].varValue > 0:
        print(f"Enviar {x[i,j].varValue:.2f} millones de galones de R{i} a A{j}")
for i in exceso:
    if exceso[i].varValue > 0:
        print(f"Excedente de R{i} enviado fuera del sistema: {exceso[i].varValue:.2f} millones")


Costo total: $ 207.0
Enviar 4.00 millones de galones de R1 a A1
Enviar 2.00 millones de galones de R1 a A2
Enviar 5.00 millones de galones de R2 a A2
Enviar 1.00 millones de galones de R3 a A2
Enviar 4.00 millones de galones de R3 a A3
Excedente de R3 enviado fuera del sistema: 3.00 millones
